## Project 1 pilot study single subject single ROI

In [15]:
# load all the packages and libraries going to be used
%pylab inline
import numpy as np
from mvpa2.suite import *
import os.path as op
import sklearn
import seaborn as sns
from pywt import wavedecn
from scipy import stats
from mpl_toolkits import mplot3d
from scipy.io import loadmat
from tqdm import tqdm


Populating the interactive namespace from numpy and matplotlib


/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['copy', 'product', 'vstack', 'deprecated', 'seed', 'random', 'ceil', 'hist', 'chisquare', 'save', 'test', 'matshow', 'hstack', 'lstsq', 'argsort', 'log10', 'info', 'floor']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
# First, let's load fMRI data, 10 runs
sub_list=["s11", "s12","s13","s14","s15","s18","s20","s21","s22","s23","s24","s25","s26","s27","s28","s29","s30","s31"]
data = []
for subjects in sub_list:

    print "subject=", subjects

    bold_fname = ('all_subjects_18'+'/'+subjects+'.nii/01.nii','all_subjects_18'+'/'+subjects+'.nii/02.nii','all_subjects_18'+'/'+subjects+'.nii/03.nii','all_subjects_18'+'/'+subjects+'.nii/04.nii','all_subjects_18'+'/'+subjects+'.nii/05.nii','all_subjects_18'+'/'+subjects+'.nii/06.nii','all_subjects_18'+'/'+subjects+'.nii/07.nii','all_subjects_18'+'/'+subjects+'.nii/08.nii','all_subjects_18'+'/'+subjects+'.nii/09.nii','all_subjects_18'+'/'+subjects+'.nii/10.nii')

    ds = fmri_dataset(bold_fname)
    data.append(ds)

subject= s11
subject= s12
subject= s13
subject= s14
subject= s15
subject= s18
subject= s20
subject= s21
subject= s22
subject= s23
subject= s24
subject= s25
subject= s26
subject= s27


In [22]:
ds.shape

(1230, 2661)

In [16]:
# Load in the mask of the ROI

VT = loadmat ("all_subjects_18/vt.mat") #load mat.file into python.
VT = VT['VT'] #get the value out of dictionary
#BA17 = loadmat ("all_subjects_18/BA17.mat")
#BA17 =BA17['BA17']

# Load in the mask of the ROI
#ds = fmri_dataset (bold_fname, mask=BA17)
ds = fmri_dataset (bold_fname, mask=VT)
print 'sample_number=',len(ds)  # check how many samples or volumes;
print'feature_number=',ds.nfeatures # check how many feature being used;
print'data_info=',ds.shape # check teh 2-dimensianl dataset info;

sample_number= 1230
feature_number= 2661
data_info= (1230, 2661)


In [18]:
# Load in the condition label file
conditions=loadmat('all_subjects_18/regressors_sh3.mat')
conditions = conditions['regressors_sh3']

In [19]:
def convert_binary_to_multiclass(binary_conditions):
    """Convert binary representation into multiclass reprentation:
    For example: convert [[1 1 1 1 0 0 0 0]
                          [0 0 0 0 1 1 1 1]]
    to [1 1 1 1 2 2 2 2]"""
    x,y = np.where(binary_conditions)
    conditions=np.zeros(binary_conditions.shape[1])
    conditions[y]=x+1
    return conditions

conditions_multi = convert_binary_to_multiclass(conditions)
conditions_multi[:123]#first run


array([ 0.,  0.,  0.,  0.,  0.,  8.,  8.,  8.,  8.,  0.,  0.,  0.,  0.,
        0.,  0.,  7.,  7.,  7.,  7.,  0.,  0.,  0.,  0.,  0.,  0., 11.,
       11., 11., 11.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  9.,  9.,  9.,
        0.,  0.,  0.,  0.,  0.,  0.,  5.,  5.,  5.,  5.,  0.,  0.,  0.,
        0.,  0.,  0., 10., 10., 10., 10.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,  3.,
        3.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.,  2.,  2.,  0.,  0.,
        0.,  0.,  0.,  0., 12., 12., 12., 12.,  0.,  0.,  0.,  0.,  0.,
        0.,  4.,  4.,  4.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  6.,
        6.,  6.,  0.,  0.,  0.,  0.])

In [20]:
# Load in runs
runs=loadmat('all_subjects_18/runs.mat')
runs = runs['runs']
runs = runs[0,:] #get rid of the rows. 

#ds = fmri_dataset (bold_fname, mask=BA17, targets= conditions_multi, chunks=runs)
ds = fmri_dataset (bold_fname, mask=VT, targets= conditions_multi, chunks=runs)
print ds.summary()

Dataset: 1230x2661@float32, <sa: chunks,targets,time_coords,time_indices>, <fa: voxel_indices>, <a: imgaffine,imghdr,imgtype,mapper,voxel_dim,voxel_eldim>
stats: mean=-3.36515e-08 std=2.99754 var=8.98523 min=-73.6473 max=88.5769

Counts of targets in each chunk:
  chunks\targets 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0
                 --- --- --- --- --- --- --- --- --- ---  ---  ---  ---
        1         75  4   4   4   4   4   4   4   4   4    4    4    4
        2         75  4   4   4   4   4   4   4   4   4    4    4    4
        3         75  4   4   4   4   4   4   4   4   4    4    4    4
        4         75  4   4   4   4   4   4   4   4   4    4    4    4
        5         75  4   4   4   4   4   4   4   4   4    4    4    4
        6         75  4   4   4   4   4   4   4   4   4    4    4    4
        7         75  4   4   4   4   4   4   4   4   4    4    4    4
        8         75  4   4   4   4   4   4   4   4   4    4    4    4
        9         75  4  

In [21]:

insect1 = ds.targets == 1
insect2 = ds.targets == 2
insect3 = ds.targets == 3
insect4 = ds.targets == 4
bird1 = ds.targets == 5
bird2 = ds.targets == 6
bird3 = ds.targets == 7
bird4 = ds.targets == 8
monkey1 = ds.targets == 9
monkey2 = ds.targets == 10
monkey3 = ds.targets == 11
monkey4 = ds.targets == 12

In [34]:
# Try different classifers-SVM
clf = LinearCSVMC()
cvte = CrossValidation(clf, NFoldPartitioner(),errorfx=lambda p, t: np.mean(p == t))
cv_results_svm= cvte(ds_new)
SVM_new = np.mean(cv_results_svm)
print 'Mean accuracy SVM=',SVM_new
print
# Test the results significance
t_new_svm, p_new_svm= stats.ttest_1samp(cv_new_results,.25)
print
print 't_new_value_svm=', t_new_svm
print 'p_new_value_svm=', p_new_svm

Mean accuracy SVM= 0.4583333333333333


t_new_value_svm= [3.38240713]
p_new_value_svm= [0.04301491]
